In [41]:
!pip install langchain langchain_openai langchain_core langchain_experimental pydantic

In [ ]:
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain_core.tools import tool
from typing import TypedDict, List, Annotated, Literal
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# LLM 초기화
llm = ChatOpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'), model="gpt-4o")
# intent_chain = LLMChain(llm=llm, prompt=intent_prompt)


# 로봇 주행 상태 목록 조회 API
@tool
def check_drive_status() -> List[TypedDict]:
    """Fetching robot driving statuses"""
    print("Fetching robot driving statuses...")
    return [{"id": "robot_1", "status": "정지"}, {"id": "robot_2", "status": "주행중"}]

# 로봇 충전 상태 목록 조회 API
@tool
def check_charge_status() -> List[TypedDict]:
    """Fetching robot charge statuses"""
    print("Fetching robot charge statuses...")
    return [{"id": "robot_1", "battery": 15}, {"id": "robot_2", "battery": 50}]

# 로봇 에러 상태 목록 조회 API
@tool
def check_error_status() -> List[TypedDict]:
    """Fetching robot error statuses"""
    print("Fetching robot error statuses...")
    return [{"id": "robot_3", "error": "battery failure"}]

# 로봇에게 충전소 이동 명령 전송 API
@tool
def move_to_charging_station(robot_id) -> str:
    """Sending robot to charging station"""
    print(f"Sending robot {robot_id} to charging station...")
    return f"Robot {robot_id} sent to charging station"

# SMS 알림 전송 API
@tool
def send_sms(message) -> str:
    """Sending SMS report"""
    print(f"Sending SMS: {message}")
    return f"SMS sent with message: {message}"


def intent_classificator(llm):
    intents= ["check_drive_status", "check_charge_status", "check_error_status", "move_to_charging_station", "send_sms"]
    system_prompt = (
        "당신은 사용자의 요청을 받고 아래에 나열된 의도들을 파악하는 의도 분류기입니다.: {intents}. "
        "사용자 요청에는 1개 또는 여러개의 의도가 존재합니다. "
        "주어진 사용자 요청에 따라, 어떤 의도를 가졌는지 순서대로 나열하세요."
    )
    prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system_prompt),
          MessagesPlaceholder(variable_name="messages"),
      ]
    ).partial(intents=str(intents))
    return prompt | llm


def tool_executor(llm):
    tools = [check_drive_status, check_charge_status, check_error_status, move_to_charging_station, send_sms]
    system_prompt = (
        "당신은 주어진 tool 실행 요청에 따라 순차적으로 tool을 실행하는 tool 실행기입니다."
        "주어진 tools : {tool_names}"
        "주어진 tool 실행 요청에 따라 tool을 순차적으로 실행하고, 결과값을 기억하세요."
        "필요시 이전 tool에서 실행한 결과값을 다음 실행할 tool에 활용하세요."
        "모든 tool이 실행이 완료되면 '실행이 완료되었습니다' 라고 응답하세요."
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("user", "{intents}"),
        ]
    ).partial(tool_names= ", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

def report_generator(llm):
  system_prompt = (
      "당신은 다른 AI들의 업무 실행 결과를 보고 사용자에게 적절한 응답을 생성해주는 AI 비서입니다."
      "AI 의 업무 실행결과를 확인하고 사용자에게 전달할 내용을 보고서 형태로 정리하세요."
  )
  prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system_prompt),
          ("user", "{result}"),
      ]
  )
  return prompt | llm


intent_classificator = intent_classificator(llm)
tool_executor = tool_executor(llm)
intent_classification_result = intent_classificator.invoke([HumanMessage(content="에러발생한 로봇들 목록 정리해서 SMS로 보내줘.")])
print(intent_classification_result.content)

tool_execution_result = tool_executor.invoke([AIMessage(content=intent_classification_result.content)])
print(tool_execution_result.content)
print(tool_execution_result.tool_calls)

report_generator = report_generator(llm)
report_result = report_generator.invoke([AIMessage(content=tool_execution_result.tool_calls)])
print(report_result.content)


['check_error_status', 'send_sms']

[{'name': 'check_error_status', 'args': {}, 'id': 'call_HOi94qfjejuX0ruwEzaYtW5H', 'type': 'tool_call'}, {'name': 'send_sms', 'args': {'message': 'Error status check initiated'}, 'id': 'call_ihp9p6dHkt09Xd9TOWAvqnLZ', 'type': 'tool_call'}]
### 보고서: AI 업무 실행 결과

#### 1. 업무 실행 요약
AI는 두 가지 주요 업무를 실행하였습니다:
1. 시스템 오류 상태 확인
2. SMS 발송

#### 2. 세부 업무 내용

**1. 시스템 오류 상태 확인**
- **업무 이름:** check_error_status
- **실행 ID:** call_HOi94qfjejuX0ruwEzaYtW5H
- **인자:** 없음
- **설명:** 시스템 오류 상태를 확인하기 위한 도구 호출

**2. SMS 발송**
- **업무 이름:** send_sms
- **실행 ID:** call_ihp9p6dHkt09Xd9TOWAvqnLZ
- **인자:**
  - **message:** Error status check initiated
- **설명:** 오류 상태 확인이 시작되었음을 알리는 SMS 발송

#### 3. 결과 요약
- 시스템 오류 상태 확인 작업을 시작하였습니다.
- "Error status check initiated"라는 메시지로 SMS 발송이 완료되었습니다.

이상입니다. 추가적인 요청 사항이나 질문이 있으면 언제든지 말씀해 주세요.
